<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Copy_of_Clean_Fasttext_10_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unidecode
!pip install word2number
!pip install contractions

     |████████████████████████████████| 235 kB 4.2 MB/s 
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=c5a037acaa1e5468eb49dfa4da9cc71270e4911d94e9a34b9c30948394af909f
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 284 kB 4.3 MB/s 
     |████████████████████████████████| 321 kB 51.0 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85440 sha256=21c7ab6e5606554e8769ff8d9f670f0557b3ca99df8b10ad60f769686483b851
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



In [5]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [72]:
dforiginal = df_fake.head(5).append(df_real.head(5))

In [73]:
df= df_fake.head(5).append(df_real.head(5))
df.reset_index(drop=True, inplace=True)

In [74]:
print(df.columns)
df.count()

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML'], dtype='object')


id           10
news_url     10
title        10
tweet_ids    10
labelML      10
dtype: int64

In [75]:
print(df)

                     id  ... labelML
0  gossipcop-2493749932  ...       0
1  gossipcop-4580247171  ...       0
2   gossipcop-941805037  ...       0
3  gossipcop-2547891536  ...       0
4  gossipcop-5476631226  ...       0
5      gossipcop-882573  ...       1
6      gossipcop-875924  ...       1
7      gossipcop-894416  ...       1
8      gossipcop-857248  ...       1
9      gossipcop-884684  ...       1

[10 rows x 5 columns]


In [76]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)
df1.count()

title      10
labelML    10
dtype: int64

In [77]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [78]:
#print(df1)
df1.count()

title      10
labelML    10
dtype: int64

In [79]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:4, 0] = df1.iloc[:, 0].apply(lambda x: '__label__0 ' + x)
df1.iloc[5:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__1 ' + x)

# NOTE BIG NOTE In general it is important to properly preprocess your data. In particular our example scripts in the root folder do this. i can see why the last cell is wierd, fix in post.


In [80]:
print(df1)

                                               title  labelML
0  __label__0 did miley cyrus and liam hemsworth ...        0
1  __label__0 paris jackson cara delevingne enjoy...        0
2  __label__0 celebrities join tax march in prote...        0
3  __label__0 cindy crawford daughter kaia gerber...        0
4             full list of oscar nominations variety        0
5  __label__1 teen mom star jenelle evans wedding...        1
6  __label__1 kylie jenner refusing to discuss ty...        1
7                           __label__1 quinn perkins        1
8  __label__1 tried kim kardashian butt workout a...        1
9  __label__1 celine dion donates concert proceed...        1


In [81]:
df1.count()

title      10
labelML    10
dtype: int64

In [44]:
df2=df1
# ------------- df2 = df2.reindex(columns=['labelML', 'title'])
#print(df1)

In [45]:
traindf, testdf = train_test_split(df2, test_size=0.2)
print(traindf)
print(testdf)  

traindf.count()

                                               title  labelML
9  __label__1 celine dion donates concert proceed...        1
4             full list of oscar nominations variety        0
7                           __label__1 quinn perkins        1
1  __label__0 paris jackson cara delevingne enjoy...        0
8  __label__1 tried kim kardashian butt workout a...        1
3  __label__0 cindy crawford daughter kaia gerber...        0
2  __label__0 celebrities join tax march in prote...        0
0  __label__0 did miley cyrus and liam hemsworth ...        0
                                               title  labelML
6  __label__1 kylie jenner refusing to discuss ty...        1
5  __label__1 teen mom star jenelle evans wedding...        1


title      8
labelML    8
dtype: int64

In [82]:
# Saving the CSV file as a text file to train/test the classifier
import csv #needed
traindf[['labelML', 'title']].to_csv('traindf.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

testdf[['labelML', 'title']].to_csv('testdf.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [83]:
!pip install fasttext
import fasttext
# Training the fastText classifier
model = fasttext.train_supervised('traindf.txt', wordNgrams = 2, dim=300)

# Evaluating performance on the entire test file
model.test('testdf.txt')                      

# Predicting on a single input
testdf['labelML'] = testdf['labelML'].astype(str)
model.predict(testdf.iloc[1, 0])
#model.test(testdf.txt)

# Save the trained model
#model.save_model('modelFTme.bin')

(('__label__1',), array([0.50001079]))

In [84]:
testdf.head()
#print(testdf.columns)

,title,labelML
6,__label__1 kylie jenner refusing to discuss ty...,1
5,__label__1 teen mom star jenelle evans wedding...,1


This model is running, so trying to change with vectors, or textual input for new vectors. Accuracy is only 50 percent coz I've given it such a balanced dataset.

In [85]:
# Predicting on a single input
#model.predict(test.iloc['Miley cyrus is new'])
#model.test(testdf.txt)
#use this later when we have the representations I guess. 


In [86]:
# Once the model is trained, we can retrieve the list of words and labels

# model.words 
model.labels
#It returns all words in the vocabulary, sorted by decreasing frequency. 

['__label__0', '__label__1']

In [87]:
# To evaluate our model by computing the precision at 1 (P@1) and the recall on a test set, we use the test function:

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('testdf.txt'))

N	2
P@1	1.000
R@1	1.000


In [88]:
#We can also predict labels for a specific text. By default, predict returns only one label :
# the one with the highest probability. You can also predict more than one label by specifying the parameter k:
model.predict("katy perry and orlando bloom cozy up", k=2)

(('__label__1', '__label__0'), array([0.50001037, 0.50000966]))

In [89]:
#traindf['title'] = 
model.get_word_vector("the")

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [90]:
model.get_sentence_vector("simon cowell rushed to hospital")

array([-6.95087074e-04, -6.53951138e-05,  1.36586474e-04, -3.16465739e-04,
        5.08174533e-04,  3.58914607e-04, -1.15762938e-04,  3.72914568e-04,
        2.42630515e-04,  3.02364584e-04,  1.38007599e-04, -5.31478363e-05,
        8.74496647e-04, -8.18732733e-05, -5.73847728e-06,  3.79808538e-04,
        1.31127686e-04,  4.12637193e-04,  2.60535540e-04, -8.57295119e-04,
       -3.03287288e-06,  6.00468949e-04,  6.74782263e-04, -4.10863606e-04,
        5.74132609e-05, -2.02935465e-04, -3.21459578e-04, -1.98773523e-05,
       -4.10947425e-04,  4.83242693e-05, -4.41215379e-04, -2.47165910e-04,
       -7.24782585e-04, -2.55656400e-04, -6.73103321e-04, -1.56693844e-04,
        1.60323456e-04, -2.47491320e-04, -9.68757740e-05, -6.35423057e-04,
        2.14422180e-04,  4.54639812e-04,  5.31330646e-04,  3.54101707e-04,
       -6.87181364e-06, -1.04621417e-04, -5.48096257e-04, -1.71012871e-04,
       -1.98763737e-04,  2.98773775e-05, -3.40820203e-04,  4.27266379e-04,
       -4.11979941e-04,  

In [91]:
model.get_nearest_neighbors('green') #https://fasttext.cc/docs/en/unsupervised-tutorial.html

[(0.0, 'of'),
 (0.0, '0'),
 (0.0, 'in'),
 (0.0, 'dining'),
 (0.0, 'with'),
 (0.0, 'after'),
 (0.0, 'wig'),
 (0.0, 'kardashian'),
 (0.0, 'chemistry'),
 (0.0, 'amazing')]

Will be attempting vectors here.

In [92]:
!pip install sister
import sister


In [ ]:
embedder = sister.MeanEmbedding(lang="en")

sentence = "I am a dog."
vector = embedder(sentence)  # 300-dim vector


Loading model...


In [ ]:
print(vector)

[ 1.45553529e-01  2.93777343e-02  7.25906417e-02 -3.32199186e-01
  1.16119541e-01  3.68818134e-01 -1.01360753e-01  6.13834858e-02
  9.52781364e-02  7.21023753e-02  2.20229030e-02 -1.12615541e-01
  5.77215105e-04  1.48535818e-01 -5.01247048e-02 -2.63762064e-02
 -1.00038767e-01  2.95178946e-02 -4.22474258e-02  6.19036630e-02
  8.73639509e-02  6.23530126e-04 -1.02920629e-01 -5.05673476e-02
 -2.43382186e-01 -5.01833446e-02  3.40687782e-02 -1.88491017e-01
  2.04292275e-02  1.45984618e-02  5.52195907e-02  8.40069652e-02
 -2.68052034e-02  1.21745087e-01 -1.02101944e-01  1.85941588e-02
 -7.98325688e-02 -2.78020445e-02 -2.32372917e-02  8.07202831e-02
  9.49704796e-02 -1.15997776e-01 -5.40836044e-02 -8.32328051e-02
  2.55872514e-02  1.02682985e-01 -1.13008637e-02 -7.92161524e-02
  5.31888008e-03 -5.63134067e-02  1.35553302e-02  2.82942466e-02
 -1.43566862e-01 -5.00163734e-02 -3.94095294e-02  7.77156502e-02
  9.01533887e-02  8.26578513e-02  1.37465566e-01  1.76570296e-01
 -1.79537944e-02  2.28038

In [93]:
dfready = dforiginal.drop(['tweet_ids', 'news_url','id'], axis=1)

from string import punctuation
from nltk.corpus import stopwords

dfready.head()


,title,labelML
0,Did Miley Cyrus and Liam Hemsworth secretly ge...,0
1,Paris Jackson & Cara Delevingne Enjoy Night Ou...,0
2,Celebrities Join Tax March in Protest of Donal...,0
3,Cindy Crawford's daughter Kaia Gerber wears a ...,0
4,Full List of 2018 Oscar Nominations – Variety,0


In [94]:
def preprocess(data):
    """
    Preprocess data, filtering out stopwords, punctuation and lowering 
    all splitted tokens.
    
    :param data: the string data to be processed
    """    
    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        data = data.replace(char, ' ' + char + ' ')
    sw = stopwords.words('english')
    splitted_chunks = data.split()
    lowered_chunks = (item.lower() for item in splitted_chunks)
    chunks_without_punctuation = (chunk for chunk in lowered_chunks if chunk not in punctuation)
    chunks_without_stopwords = (chunk for chunk in chunks_without_punctuation if chunk not in sw)
    return " ".join(list(chunks_without_stopwords))


In [95]:
import pandas as pd

def create_vectors(title):
  embedder = sister.MeanEmbedding(lang="en")
  sentence = title
  vector = embedder(sentence)  # 300-dim vector
  return vector

# executing the function
dfready['title'] = dfready['title'].apply(preprocess)

# executing the function
dfready['vectorcol'] = dfready['title'].apply(create_vectors)


Loading model...


Loading model...
Loading model...


Loading model...
Loading model...


Loading model...
Loading model...


Loading model...


Loading model...
Loading model...


In [96]:
dfready.head()

,title,labelML,vectorcol
0,miley cyrus liam hemsworth secretly get married,0,"[-0.12480343, -0.06694698, -0.18565401, 0.1694..."
1,paris jackson cara delevingne enjoy night matc...,0,"[0.08161997, 0.019448318, -0.05987478, -0.1275..."
2,celebrities join tax march protest donald trump,0,"[-0.0071617686, 0.05993736, 0.22043763, -0.147..."
3,cindy crawford's daughter kaia gerber wears wi...,0,"[0.051251747, 0.003807944, -0.023948468, 0.009..."
4,full list 2018 oscar nominations – variety,0,"[-0.04029811, -0.0011204657, 0.2647286, -0.059..."


In [108]:
dfready[['vectorcol']]

,vectorcol
0,"[-0.12480343, -0.06694698, -0.18565401, 0.1694..."
1,"[0.08161997, 0.019448318, -0.05987478, -0.1275..."
2,"[-0.0071617686, 0.05993736, 0.22043763, -0.147..."
3,"[0.051251747, 0.003807944, -0.023948468, 0.009..."
4,"[-0.04029811, -0.0011204657, 0.2647286, -0.059..."
0,"[-0.07722194, -0.05114095, -0.06775734, -0.041..."
1,"[0.03420729, -0.07510132, -0.020634918, -0.002..."
2,"[-0.30353433, -0.20470504, -0.12673381, -0.285..."
3,"[0.026270678, 0.09265323, -0.10167308, 0.08931..."
4,"[0.023988368, 0.038268402, -0.10333933, -0.012..."


In [112]:
X = dfready[['vectorcol']]

In [113]:
X

,vectorcol
0,"[-0.12480343, -0.06694698, -0.18565401, 0.1694..."
1,"[0.08161997, 0.019448318, -0.05987478, -0.1275..."
2,"[-0.0071617686, 0.05993736, 0.22043763, -0.147..."
3,"[0.051251747, 0.003807944, -0.023948468, 0.009..."
4,"[-0.04029811, -0.0011204657, 0.2647286, -0.059..."
0,"[-0.07722194, -0.05114095, -0.06775734, -0.041..."
1,"[0.03420729, -0.07510132, -0.020634918, -0.002..."
2,"[-0.30353433, -0.20470504, -0.12673381, -0.285..."
3,"[0.026270678, 0.09265323, -0.10167308, 0.08931..."
4,"[0.023988368, 0.038268402, -0.10333933, -0.012..."


In [142]:
type(X)

pandas.core.frame.DataFrame

In [143]:
y = dfready[['labelML']]

In [144]:
type(y)

pandas.core.frame.DataFrame

In [160]:
X_train.dtypes
#X_train = X_train.drop(['array'])

vectorcol    object
dtype: object

In [161]:
X_train_new = X_train.to_numpy()
y_train_new = y_train.to_numpy()
X_test_new = X_test.to_numpy()
y_test_new = y_test.to_numpy()

In [164]:
#X_train_new = X_train_new.drop['array']
X_train_new.shape
#type(X_train_new)
#type(y_train_new)

(9, 1)

In [156]:
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

ValueError: ignored

In [120]:
import xgboost as xgb
#TASK #2: Create an XGBoost classifier that will work for the vectors that you have (size 300), and train it....

#X, y = dfready.[:, 2:], dfready['labelML']
#print(X.columns)

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)



In [105]:
print(X_train)

                                           vectorcol
0  [-0.12480343, -0.06694698, -0.18565401, 0.1694...
2  [-0.30353433, -0.20470504, -0.12673381, -0.285...
0  [-0.07722194, -0.05114095, -0.06775734, -0.041...
3  [0.026270678, 0.09265323, -0.10167308, 0.08931...
3  [0.051251747, 0.003807944, -0.023948468, 0.009...
1  [0.08161997, 0.019448318, -0.05987478, -0.1275...
1  [0.03420729, -0.07510132, -0.020634918, -0.002...
4  [0.023988368, 0.038268402, -0.10333933, -0.012...
2  [-0.0071617686, 0.05993736, 0.22043763, -0.147...


In [139]:
# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train_new, y_train_new)


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [ ]:

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

#TASK 3: Analyze the results of one training example with the confusion matrix

#If you can also get the confusion matrix